In [1]:
%cd ..

/home/eli/AnacondaProjects/ppc_experiments


In [2]:
import argparse
import collections
import numpy as np
import pyro
import torch
import data_loader.data_loaders as module_data
import model.loss as module_loss
import model.metric as module_metric
import model.model as module_arch
from parse_config import ConfigParser
import trainer.trainer as module_trainer

In [3]:
# pyro.enable_validation(True)
# torch.autograd.set_detect_anomaly(True)

In [4]:
# fix random seeds for reproducibility
SEED = 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [5]:
def main(config):
    logger = config.get_logger('train')

    # setup data_loader instances
    data_loader = config.init_obj('data_loader', module_data)
    valid_data_loader = data_loader.split_validation()

    # build model architecture, then print to console
    model = config.init_obj('arch', module_arch)
    logger.info(model)

    # get function handles of metrics
    metrics = [getattr(module_metric, met) for met in config['metrics']]

    # build optimizer.
    optimizer = config.init_obj('optimizer', pyro.optim)

    # build trainer
    # kwargs = config['trainer'].pop('args')
    trainer = config.init_obj('trainer', module_trainer, model, metrics, optimizer,
                              config=config, data_loader=data_loader,
                              valid_data_loader=valid_data_loader,
                              lr_scheduler=None)

    return trainer

In [6]:
from utils import read_json

config = read_json("experiments/ppc_bmnist_config.json")
config = ConfigParser(config)
trainer = main(config)
trainer.train()

BouncingMnistPpc(
  (decoder): DigitsDecoder(
    (decoder): Sequential(
      (0): Linear(in_features=10, out_features=200, bias=True)
      (1): ReLU()
      (2): Linear(in_features=200, out_features=400, bias=True)
      (3): ReLU()
      (4): Linear(in_features=400, out_features=784, bias=True)
      (5): Sigmoid()
    )
  )
  (digit_features): DigitFeatures()
  (digit_positions): DigitPositions()
  (graph): GraphicalModel()
)
Trainable parameters: 396984
Initialize particles: train batch 0
Initialize particles: train batch 1
Initialize particles: train batch 2
Initialize particles: train batch 3
Initialize particles: train batch 4
Initialize particles: train batch 5
Initialize particles: train batch 6
Initialize particles: train batch 7
Initialize particles: train batch 8
Initialize particles: train batch 9
Initialize particles: train batch 10
Initialize particles: train batch 11
Initialize particles: train batch 12
Initialize particles: train batch 13
Initialize particles: train 

AssertionError: ParticleDict

from user code:
   File "/home/eli/AnacondaProjects/ppc_experiments/trainer/trainer.py", line 207, in _ppc_step
    self._load_particles(batch_indices, train)
  File "/home/eli/AnacondaProjects/ppc_experiments/trainer/trainer.py", line 192, in _load_particles
    for site in particles:

Set TORCH_LOGS="+dynamo" and TORCHDYNAMO_VERBOSE=1 for more information


You can suppress this exception and fall back to eager by setting:
    import torch._dynamo
    torch._dynamo.config.suppress_errors = True


In [ ]:
trained.model.eval()
trained.model.cpu()